Homework 2
=====

Ramsey Villarreal<br/>
CSC 576 - Data Mining<br/>
Dr. Burns<br/>
March 5, 2019

Question 1
-----------
#### Data.gov
Find the nth most recent article from http://catalog.data.gov/dataset?q=&sort=metadata_created+desc <br/>
Where n = integer input from the user

In [1]:
import bs4
import math
import requests
import ipywidgets as widgets
import tweepy
import pandas as pd

In [2]:
response = requests.get('http://www.data.gov/')
soup = bs4.BeautifulSoup(response.text,"html.parser")
link = soup.select("small a")[0]
print(link.text)

246,194 datasets


In [3]:
def get_dataset_at(index=1):
    response = requests.get('http://catalog.data.gov/dataset?q=&sort=metadata_created+desc')
    soup = bs4.BeautifulSoup(response.text,"html.parser")
    dataset_items = soup.findAll("h3", {"class":"dataset-heading"})
    
    num_on_page = len(dataset_items)
    
    #if the dataset is not on the first page, get page that it is on 
    if(index>num_on_page):
        page = math.ceil(index/num_on_page)
        new_url = 'http://catalog.data.gov/dataset?q=&sort=metadata_created+desc&page='+str(page)
        response = requests.get(new_url)
        soup = bs4.BeautifulSoup(response.text,"html.parser")
        dataset_items = soup.findAll("h3", {"class":"dataset-heading"})
#       reset index to correspond to dataset on page
        index = index%num_on_page 

    return dataset_items[index-1].text

In [4]:
#find number of datasets, return string that can be typecasted to int
num_of_datasets = ""
for i in link.text:
    try:
        if(int(i) or i=="0"):
            num_of_datasets += i
    except ValueError:
        continue
print("There are " + link.text )
# print(num_of_datasets)


#get user input - only int between 1 and num_of_datasets
while True:
    try:
        num = int(input("Which dataset would you like to view? (starting at 1): "))
        
        if(num<1 or num>int(num_of_datasets)):
            raise ValueError
        break
    except ValueError:
         print("Invalid integer. The number must be in the range of 1-" + link.text)
print(get_dataset_at(int(num)))

There are 246,194 datasets
Which dataset would you like to view? (starting at 1): 21

Admissions to Maryland State-Supported Alcohol and Drug Abuse Treatment Programs by Patient Residence, SFY 2008-2012





## Question 2

#### White House Briefings

Find the most recent press conference from  https://www.whitehouse.gov/briefing-room/press-briefings.
It will be identified by the "Remarks" subheader 

In [5]:
def get_most_recent_remarks():
    n=0
    
    # Go through all of the pages until the most recent remarks is found
    while True:
        n += 1
        url = 'https://www.whitehouse.gov/briefings-statements/page/' + str(n) + '/'
        response = requests.get(url)
        soup = bs4.BeautifulSoup(response.text,"html.parser")
        # Find all of the articles on the page
        briefs = soup.findAll("article", {"class":"briefing-statement"})


        for i in briefs:
            brief_type = i.find("p",{"class":"briefing-statement__type"})

            # if brief is not none and if Remarks is the type then it is the most recent remarks
            if brief_type:
                if "Remarks" in brief_type.text:
                    
                    # Find the link to the article and parse that
                    remarks_url = i.find("a")["href"]
                    response = requests.get(remarks_url)
                    remarks_soup = bs4.BeautifulSoup(response.text,"html.parser")
                    
                    # Find the time issued in the article
                    all_text = remarks_soup.findAll("p")
                    times = []
                    for text in all_text:
        
                        if "A.M." in text.text or "P.M." in text.text:
                            times.append(text)
                        
                            
                    # print remarks information
                    print("Most recent Remarks: " + i.find("a").text + 
                          "\nURL: " + remarks_url +
                          "\nIssued on: " + i.find("time").text + " " + times[0].text)
                    
                    return

In [6]:
get_most_recent_remarks()

Most recent Remarks: Remarks by President Trump at Lima Army Tank Plant
URL: https://www.whitehouse.gov/briefings-statements/remarks-president-trump-lima-army-tank-plant/
Issued on: Mar 20, 2019 3:15 P.M. EDT


## Question 3

#### Texas Dept of Criminal Justice
Accept a starting and ending year to scrape  http://www.tdcj.state.tx.us/death_row/dr_executions_by_year.html to find all executions between and including those years

In [7]:
def find_all_executions_between(table,starting_year, ending_year):
    total_deaths = 0
    for row in table[1:]:
        year = row.find("th",{"scope":"row"})
#         print(year.text)
        if year:
            try:
                y = int(year.text)
                if(y>=starting_year and y<=ending_year):
#                     print(row.findAll("td")[-2].text)
                    deaths = int(row.findAll("td")[-2].text)
                    total_deaths += deaths
            except ValueError:
                continue
    print("Total executions between and including " + str(starting_year) + " and " + str(ending_year) + ": " + str(total_deaths))
    return

In [8]:
def find_year_range(table):
    years = []
    
    for i, year in enumerate(table):
        y = year.find("th",{"scope":"row"})
        if y:
            try:
                y = int(y.text)
                years.append(y)
            except ValueError:
                continue

    return years

In [16]:

url = 'http://www.tdcj.state.tx.us/death_row/dr_executions_by_year.html'
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text,"html.parser")
table = soup.findAll("tr")
# print(len(table))
years = find_year_range(table)

w = widgets.IntRangeSlider(
        value=[min(years), max(years)],
        min= min(years),
        max= max(years),
        step=1,
        description='Years:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
    )
display(w)
button = widgets.Button(description="Find Total Executions")
display(button)


def on_button_clicked(b):
    find_all_executions_between(table,w.value[0], w.value[1])
    

button.on_click(on_button_clicked)

Total executions between and including 1982 and 2006: 379
Total executions between and including 1982 and 2006: 379


IntRangeSlider(value=(1982, 2019), continuous_update=False, description='Years:', max=2019, min=1982)

Button(description='Find Total Executions', style=ButtonStyle())

Total executions between and including 1982 and 2009: 447


# Question 4

#### Senator Twitter

Using the data from http://unitedstates.sunlightfoundation.com/legislators/legislators.csv
    - Find out who has the most Twitter followers
    - Find out who has the fewest Twitter followers
    - Look up the 10 most recent tweets for each senator
        - Find how many people retweeted them
        - Find how many people liked them

In [10]:
url = "http://unitedstates.sunlightfoundation.com/legislators/legislators.csv"
df = pd.read_csv(url)

#Keep only Senators that are still active and that have twitter ids
active_senators = df.loc[(df['title'] == "Sen") & (df['in_office']==1)]
active_senators = active_senators.dropna(subset=["twitter_id"])
active_senators

,title,firstname,middlename,lastname,name_suffix,nickname,party,state,district,in_office,...,govtrack_id,crp_id,twitter_id,congresspedia_url,youtube_url,facebook_id,official_rss,senate_class,birthdate,oc_email
8,Sen,Lamar,NaN,Alexander,NaN,NaN,R,TN,Senior Seat,1,...,300002,N00009888,SenAlexander,http://www.opencongress.org/wiki/Lamar_Alexander,http://youtube.com/lamaralexander,senatorlamaralexander,NaN,II,1940-07-03,Sen.Alexander@opencongress.org
38,Sen,Roy,NaN,Blunt,NaN,NaN,R,MO,Junior Seat,1,...,400034,N00005195,RoyBlunt,http://www.opencongress.org/wiki/Roy_Blunt,http://youtube.com/SenatorBlunt,SenatorBlunt,NaN,III,1950-01-10,Sen.Blunt@opencongress.org
47,Sen,Sherrod,NaN,Brown,NaN,NaN,D,OH,Senior Seat,1,...,400050,N00003535,SenSherrodBrown,http://www.opencongress.org/wiki/Sherrod_Brown,http://youtube.com/SherrodBrownOhio,SenatorSherrodBrown,NaN,I,1952-11-09,Sen.Brown@opencongress.org
50,Sen,Richard,M.,Burr,NaN,NaN,R,NC,Senior Seat,1,...,400054,N00002221,SenatorBurr,http://www.opencongress.org/wiki/Richard_Burr,http://youtube.com/SenatorRichardBurr,SenatorRichardBurr,NaN,III,1955-11-30,Sen.Burr@opencongress.org
57,Sen,Tammy,NaN,Baldwin,NaN,NaN,D,WI,Junior Seat,1,...,400013,N00004367,SenatorBaldwin,http://www.opencongress.org/wiki/Tammy_Baldwin,http://youtube.com/witammybaldwin,senatortammybaldwin,NaN,I,1962-02-11,Sen.Baldwin@opencongress.org
63,Sen,John,NaN,Boozman,NaN,NaN,R,AR,Senior Seat,1,...,400040,N00013873,JohnBoozman,http://www.opencongress.org/wiki/John_Boozman,http://youtube.com/BoozmanPressOffice,JohnBoozman,NaN,III,1950-12-10,Sen.Boozman@opencongress.org
82,Sen,John,A.,Barrasso,NaN,NaN,R,WY,Junior Seat,1,...,412251,N00006236,SenJohnBarrasso,http://www.opencongress.org/wiki/John_Barrasso,http://youtube.com/barrassowyo,johnbarrasso,NaN,I,1952-07-21,Sen.Barrasso@opencongress.org
88,Sen,Michael,F.,Bennet,NaN,NaN,D,CO,Senior Seat,1,...,412330,N00030608,SenBennetCo,http://www.opencongress.org/wiki/Michael_Bennet,http://youtube.com/SenatorBennet,senbennetco,NaN,III,1964-11-28,Sen.Bennet@opencongress.org
98,Sen,Richard,NaN,Blumenthal,NaN,NaN,D,CT,Senior Seat,1,...,412490,N00031685,SenBlumenthal,http://www.opencongress.org/wiki/Richard_Blume...,http://youtube.com/SenatorBlumenthal,SenBlumenthal,NaN,III,1946-02-13,Sen.Blumenthal@opencongress.org
108,Sen,Cory,Anthony,Booker,NaN,NaN,D,NJ,Junior Seat,1,...,412598,N00035267,SenBooker,NaN,http://youtube.com/SenCoryBooker,NaN,NaN,II,1969-04-27,Sen.Booker@opencongress.org


In [11]:
#Get keys and access tokens from json file
import json

with open('twitterCredentials.json') as f:
    data = json.load(f)

consumer_key = data['consumer_key']
consumer_secret = data['consumer_secret']
access_token = data['access_token']
access_token_secret = data['access_token_secret']

In [12]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)


In [13]:
#Create columns for twitter information
active_senators['follower_count'] = 0
active_senators['last_10_retweets'] = 0
active_senators['last_10_favorites'] = 0

#Iterate over active senators
for index, row in active_senators.iterrows():
    twitter_id = row['twitter_id']
    
    try:
        sen = api.get_user(twitter_id)
        active_senators.loc[index,'follower_count'] = sen.followers_count

        public_tweets = api.user_timeline(twitter_id)

        #Collect data about last 10 tweets - add it to proper column
        for tweet in public_tweets[:10]:
            active_senators.loc[index,'last_10_retweets'] +=  tweet.retweet_count
            active_senators.loc[index,'last_10_favorites'] += tweet.favorite_count
            
    #If error is raised, assume user does not have twitter account and delete from dataframe
    except tweepy.TweepError:
        print(str(index) + ". " + twitter_id)
        active_senators = active_senators.drop([index])

88. SenBennetCo
500. SenKaineOffice


In [14]:
#Sort the senators by follower count
active_senators = active_senators.sort_values('follower_count')

In [15]:
print("Most Followers: " + active_senators.iloc[[-1]]['firstname'].to_string(index=False) +
     " " +active_senators.iloc[[-1]]['lastname'].to_string(index=False) +
     " @" + active_senators.iloc[[-1]]['twitter_id'].to_string(index=False) +
     " - " + active_senators.iloc[[-1]]['follower_count'].to_string(index=False))

print("Least Followers: " + active_senators.iloc[[0]]['firstname'].to_string(index=False) +
     " " +active_senators.iloc[[0]]['lastname'].to_string(index=False) +
     " @" + active_senators.iloc[[0]]['twitter_id'].to_string(index=False) +
     " - " + active_senators.iloc[[0]]['follower_count'].to_string(index=False))

print("\n")

for index, row in active_senators.iterrows():
    print(row['firstname'] + " " + row['lastname'] + " last 10 tweets: " + 
          str(row['last_10_favorites']) + " favorites, " + 
          str(row['last_10_retweets']) + " retweets." )

Most Followers: Bernard Sanders @SenSanders - 8245281
Least Followers: Christopher Murphy @senmurphyoffice - 3551


Christopher Murphy last 10 tweets: 11 favorites, 6 retweets.
Luther Strange last 10 tweets: 296 favorites, 71 retweets.
Chris Coons last 10 tweets: 82 favorites, 55 retweets.
James Inhofe last 10 tweets: 7 favorites, 68 retweets.
Claire McCaskill last 10 tweets: 740 favorites, 303 retweets.
Mike Rounds last 10 tweets: 262 favorites, 83 retweets.
Catherine Cortez Masto last 10 tweets: 222 favorites, 38 retweets.
Dan Sullivan last 10 tweets: 321 favorites, 73 retweets.
Todd Young last 10 tweets: 278 favorites, 74 retweets.
Steve Daines last 10 tweets: 580 favorites, 418 retweets.
Brian Schatz last 10 tweets: 2277 favorites, 675 retweets.
John Kennedy last 10 tweets: 1939 favorites, 850 retweets.
Joni Ernst last 10 tweets: 812 favorites, 181 retweets.
David Perdue last 10 tweets: 1794 favorites, 569 retweets.
John Isakson last 10 tweets: 329 favorites, 77 retweets.
Shelley C